# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

In [1]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm

In [2]:
data = pd.read_csv("../data/hulls_df_compact_matchday1.csv")
data = data.dropna()
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,561.031976,1195.207383
1,3.0,1.0,496.093264,1115.390332
2,4.0,1.0,466.225606,1083.424411
3,5.0,1.0,503.593453,1105.121644
4,6.0,1.0,563.030125,1132.199093


**NOTA:** provare a scalare i dati per vedere:
1. se il modello fitta più veloce
2. se i parametri finali sono più sensati

In [12]:
sequence = torch.tensor(data["AwayHull"].values)
hidden_dim = 2
sequence.shape

torch.Size([3911])

In [6]:
def model(sequence, hidden_dim, include_prior=True):
    length = len(sequence)
    with poutine.mask(mask=include_prior):
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(),
        )
        # Emission probabilities (1-dimensional for the area)
        probs_alpha = pyro.sample(
            "probs_alpha",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )

        probs_beta = pyro.sample(
            "probs_beta",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )
    
    x = 0  # Initial hidden state
    for t in pyro.markov(range(length)):
        x = pyro.sample(
            f"x_{t}",
            dist.Categorical(probs_x[x]),
            infer={"enumerate": "parallel"},
        )
        pyro.sample(
            f"y_{t}",
            dist.Gamma(probs_alpha[x], probs_beta[x]),
            obs=sequence[t],
        )
# Define the guide (variational distribution)
guide = AutoDelta(poutine.block(model, expose=["probs_x", "probs_alpha", "probs_beta"]))

---

**⚠️ DANGER ZONE ⚠️**

In [7]:
pyro.clear_param_store()

---

In [14]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Training
num_steps = 1000
tqdm_bar = tqdm.tqdm(range(num_steps))
for step in tqdm_bar:
    loss = svi.step(sequence, hidden_dim)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 1000/1000 [38:17<00:00,  2.30s/it, LOSS=2.76e+4]


In [15]:
posterior = guide(sequence,hidden_dim)
# can i save posterior?
posterior

{'probs_x': tensor([[0.9672, 0.0328],
         [0.0506, 0.9494]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([ 6.5119, 35.4986], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.0058, 0.0493], grad_fn=<ExpandBackward0>)}

In [16]:
torch.save(posterior,"parameters/singleHMM_away.pt")

In [18]:
mean_state1=6.5119/0.0058
print(f">> Mean of the Convex Hull for home team (STATE 1): {mean_state1:.2f} m^2")

>> Mean of the Convex Hull for home team (STATE 1): 1122.74 m^2


In [19]:
mean_state2=35.4986/0.0493
print(f">> Mean of the Convex Hull for home team (STATE 2): {mean_state2:.2f} m^2")

>> Mean of the Convex Hull for home team (STATE 2): 720.05 m^2
